<a href="https://colab.research.google.com/github/dain1403/Deep-Learning-with-Colab/blob/master/33_Cifar_10_Keras_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Activation, BatchNormalization
from tensorflow.keras.regularizers import l2
import numpy as np

#### 자료형 변환 및 스케일링
- X: 실수형으로 정규화
- Y: 1-hot encoding
    * airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck

In [ ]:
(X_train, y_train0), (X_test, y_test0) = cifar10.load_data()
print(X_train.shape, X_train.dtype)
print(y_train0.shape, y_train0.dtype)
print(X_test.shape, X_test.dtype)
print(y_test0.shape, y_test0.dtype)

(50000, 32, 32, 3) uint8
(50000, 1) uint8
(10000, 32, 32, 3) uint8
(10000, 1) uint8


In [ ]:
X_train = X_train.astype('float32')/255.0
X_test = X_test.astype('float32')/255.0

print(X_train.shape, X_train.dtype)

(50000, 32, 32, 3) float32


In [ ]:
Y_train = tf.keras.utils.to_categorical(y_train0, 10)
Y_test = tf.keras.utils.to_categorical(y_test0, 10)
Y_train[:4]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

#### 모형 구현

In [ ]:
np.random.seed(0)
weight_decay = 1e-4

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(weight_decay), 
                 input_shape=X_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy', 
              optimizer=RMSprop(lr=0.001, decay=weight_decay), 
              metrics=['accuracy'])

In [ ]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 30:
        lrate = 0.0005
    if epoch > 40:
        lrate = 0.0003
    return lrate

In [ ]:
#data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(X_train)

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler

history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=64),
                    steps_per_epoch=X_train.shape[0] // 64, epochs=50,
                    verbose=2, validation_data=(X_test, Y_test),
                    callbacks=[LearningRateScheduler(lr_schedule)])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
781/781 - 23s - loss: 1.9281 - accuracy: 0.4193 - val_loss: 1.2871 - val_accuracy: 0.5772 - lr: 0.0010
Epoch 2/50
781/781 - 23s - loss: 1.3111 - accuracy: 0.5816 - val_loss: 1.0588 - val_accuracy: 0.6678 - lr: 0.0010
Epoch 3/50
781/781 - 23s - loss: 1.0972 - accuracy: 0.6478 - val_loss: 1.1228 - val_accuracy: 0.6648 - lr: 0.0010
Epoch 4/50
781/781 - 23s - loss: 0.9890 - accuracy: 0.6851 - val_loss: 0.9208 - val_accuracy: 0.7210 - lr: 0.0010
Epoch 5/50
781/781 - 22s - loss: 0.9159 - accuracy: 0.7131 - val_loss: 0.8879 - val_accuracy: 0.7302 - lr: 0.0010
Epoch 6/50
781/781 - 23s - loss: 0.8615 - accuracy: 0.7299 - val_loss: 0.8758 - val_accuracy: 0.7344 - lr: 0.0010
Epoch 7/50
781/781 - 23s - loss: 0.8231 - accuracy: 0.7473 - val_loss: 0.7926 - val_accuracy: 0.7690 - lr: 0.0010
Epoch 8/50
781/781 - 22s - loss: 0.7838 - accuracy: 0.7601 - val_loss: 0.7925 - val_accuracy: 0.7685 - lr: 0.0010
Epoch 9/50
7

##### 모델 평가하기

In [ ]:
scores = model.evaluate(X_test, Y_test, batch_size=128, verbose=2)
print('\nAccuracy: %.4f' % scores[1])

79/79 - 0s - loss: 0.4679 - accuracy: 0.8719

Accuracy: 0.8719


###### 모델 저장하기